<a href="https://colab.research.google.com/github/mmilannaik/bostonhousepricing/blob/main/W40S5_MiniBatchClustering_NYC_YellowTaxi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
elemento_nyc_yellow_taxi_trip_data_path = kagglehub.dataset_download('elemento/nyc-yellow-taxi-trip-data')

print('Data source import complete.')


100%|██████████| 1.78G/1.78G [00:25<00:00, 75.6MB/s]

Extracting files...


Data source import complete.


In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans

In [8]:
data = pd.read_csv("/kaggle/input/nyc-yellow-taxi-trip-data/yellow_tripdata_2016-01.csv")
data.shape

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/nyc-yellow-taxi-trip-data/yellow_tripdata_2016-01.csv'

In [5]:
data.head()

NameError: name 'data' is not defined

In [ ]:
data.isna().sum()

## 2.Pre-Processing

In [ ]:

# Load the data
# data = pd.read_csv('/kaggle/input/nyc-yellow-taxi-trip-data/yellow_tripdata_2016-01.csv')

# Convert Y/N to 1/0
data['store_and_fwd_flag'] = LabelEncoder().fit_transform(data['store_and_fwd_flag'])

# Convert timestamp columns to datetime
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'])
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'])

# Extract features from timestamp columns (hour, day, month)
data['pickup_hour'] = data['tpep_pickup_datetime'].dt.hour
data['pickup_day'] = data['tpep_pickup_datetime'].dt.day
data['pickup_month'] = data['tpep_pickup_datetime'].dt.month
data['dropoff_hour'] = data['tpep_dropoff_datetime'].dt.hour
data['dropoff_day'] = data['tpep_dropoff_datetime'].dt.day
data['dropoff_month'] = data['tpep_dropoff_datetime'].dt.month

# Drop unnecessary columns
data.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'], axis=1, inplace=True)

# Standardize numerical features
numerical_features = ['trip_distance', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount']
data[numerical_features] = StandardScaler().fit_transform(data[numerical_features])

# Display the preprocessed data
data.head()

In [ ]:
data.payment_type.value_counts()

## Determine No of Clusters

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
# Select all features for clustering
features_for_clustering = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                               'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax',
                               'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'pickup_hour', 'pickup_day',
       'pickup_month', 'dropoff_hour', 'dropoff_day', 'dropoff_month']
len(features_for_clustering)

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score
import numpy as np
import pandas as pd

# Deciding no of clusters upon 10Lakhs Sample
X = data[features_for_clustering].sample(100000)

# Assuming 'data' is your pre-processed DataFrame
n_samples = len(X)
n_clusters_range = range(2, 11)  # Number of clusters range from 2 to 10

# Specify the number of chunks
n_chunks = 10
chunk_size = n_samples // n_chunks

# Placeholder for silhouette scores
silhouette_scores = []
inertia = []

# Loop through different cluster numbers
for n_clusters in n_clusters_range:
    # Initialize MiniBatchKMeans
    mbkmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, n_init='auto', batch_size=10000)

    # Loop through chunks
    for i in range(n_chunks):
        # Determine start and end index for the chunk
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if i < n_chunks - 1 else n_samples

        # Extract the chunk
        chunk = X.iloc[start_idx:end_idx]


        # Partially fit the model on the chunk
        mbkmeans.partial_fit(chunk)


    # Predict clusters for all data points
    labels = mbkmeans.predict(X)

    # Inertia
    inertia.append(mbkmeans.inertia_)
    # Calculate silhouette score
    silhouette = silhouette_score(X, labels)
    silhouette_scores.append(silhouette)
    print(f"Training for No of Clusters- {n_clusters} completed!!")

In [ ]:
import matplotlib.pyplot as plt

# Plot the silhouette scores
plt.plot(n_clusters_range, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score for Different Numbers of Clusters')
plt.show()

* As per silhouette, No of Clusters = 2

In [ ]:
# Plot inertia vs No Of centroids
plt.plot(n_clusters_range, inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Inertia for Different Numbers of Clusters')
plt.show()

* As per Elbow method no of cluster: 5

In [ ]:
inertia

In [ ]:
silhouette_scores

In [ ]:
# Taking optimal no of clusters as 5

X = data[features_for_clustering]

n_samples = len(X)

# Specify the number of chunks
n_chunks = 10
chunk_size = n_samples // n_chunks

# Initialize MiniBatchKMeans
mbkmeans = MiniBatchKMeans(n_clusters=5, random_state=42, n_init='auto', batch_size=chunk_size)

# Loop through chunks
for i in range(n_chunks):
    # Determine start and end index for the chunk
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if i < n_chunks - 1 else n_samples

    # Extract the chunk
    chunk = X.iloc[start_idx:end_idx]

    # Partially fit the model on the chunk
    mbkmeans.partial_fit(chunk)

# Predict clusters for all data points
data['labels'] = mbkmeans.predict(X)

In [ ]:
data['labels'].value_counts()